In [11]:
import numpy as np
import os
import re
import ast
from sklearn.linear_model import LinearRegression

# =========================
# CONFIG
# =========================
base_path = "data/"             # ganti sesuai lokasi folder function_1 ... function_8
inputs_txt = "week1/inputs.txt"       # file dari email (queries minggu lalu)
outputs_txt = "week1/outputs.txt"     # file dari email (outputs minggu lalu)
n_candidates = 1000             # jumlah kandidat acak untuk dipilih yang terbaik

# =========================
# HELPERS
# =========================
def load_outputs(path):
    """Load outputs.txt in either [a,b,c] or newline form."""
    with open(path, "r") as f:
        text = f.read().strip()
    # Coba ast.literal_eval langsung
    try:
        data = ast.literal_eval(text)
        return np.array(data, dtype=float)
    except Exception:
        # Fallback: ambil semua angka float dengan regex
        nums = re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", text)
        return np.array([float(x) for x in nums], dtype=float)

def load_inputs(path):
    """
    Load inputs.txt yang mungkin berformat:
    - [array([0.1, 0.2]), array([0.3, 0.4, 0.5]), ...]
    - [[0.1, 0.2], [0.3, 0.4, 0.5], ...]
    - JSON serupa
    Return: list of np.array, panjang 8 (satu per fungsi).
    """
    with open(path, "r") as f:
        text = f.read().strip()

    # 1) Jika ada 'array(' → parse tiap vector di dalam array(...)
    if "array(" in text:
        inner = re.findall(r"array\(\s*\[([^\]]+)\]\s*\)", text, flags=re.MULTILINE)
        vectors = []
        for s in inner:
            # split angka, buang spasi/koma kosong
            parts = [p.strip() for p in s.replace("\n", " ").split(",") if p.strip() != ""]
            vec = np.array([float(p) for p in parts], dtype=float)
            vectors.append(vec)
        if len(vectors) == 0:
            raise ValueError("Could not parse any vectors from inputs.txt containing 'array('.")
        return vectors

    # 2) Coba JSON / literal list of lists
    try:
        data = ast.literal_eval(text)
        # data harus iterable berisi vektor
        vectors = [np.array(row, dtype=float) for row in data]
        return vectors
    except Exception:
        pass

    # 3) Fallback keras: cari semua bracket level-1 berisi angka
    inner = re.findall(r"\[([0-9eE\.\,\s\-]+)\]", text)
    vectors = []
    for s in inner:
        parts = [p.strip() for p in s.replace("\n", " ").split(",") if p.strip() != ""]
        if len(parts) == 0:
            continue
        vec = np.array([float(p) for p in parts], dtype=float)
        vectors.append(vec)

    if len(vectors) == 0:
        raise ValueError("Unsupported inputs.txt format. Please show a snippet of the file.")
    # Kemungkinan ini juga menangkap list terluar; filter dengan panjang vektor unik
    # Kita ambil hanya vektor dengan panjang <= 8 (maks dim yang kita tahu)
    vectors = [v for v in vectors if 1 <= len(v) <= 8]
    # Pastikan hanya 8 vektor terakhir (satu per fungsi)
    if len(vectors) >= 8:
        return vectors[-8:]
    return vectors

# =========================
# LOAD WEEK-1 DATA FROM EMAIL
# =========================
new_inputs = load_inputs(inputs_txt)     # list of 8 arrays
new_outputs = load_outputs(outputs_txt)  # array shape (8,)

if len(new_inputs) != 8 or len(new_outputs) != 8:
    raise ValueError(f"Expect 8 inputs and 8 outputs, got {len(new_inputs)} inputs and {len(new_outputs)} outputs.")

print(f"Loaded {len(new_inputs)} Week-1 input queries and {len(new_outputs)} outputs from email.")

# =========================
# PROCESS EACH FUNCTION
# =========================
queries_out = []  # simpan string untuk file output

for i in range(1, 9):
    folder = os.path.join(base_path, f"function_{i}")
    X_init = np.load(os.path.join(folder, "initial_inputs.npy"))
    y_init = np.load(os.path.join(folder, "initial_outputs.npy"))

    # Gabung dengan query & output minggu lalu (email)
    X_combined = np.vstack([X_init, new_inputs[i-1].reshape(1, -1)])
    y_combined = np.append(y_init, new_outputs[i-1])

    # Simpan versi updated (11 sampel sekarang)
    np.save(os.path.join(folder, "updated_inputs.npy"), X_combined)
    np.save(os.path.join(folder, "updated_outputs.npy"), y_combined)

    dim = X_combined.shape[1]
    print(f"\n=== Function {i} ({dim}D) ===")
    print(f"  Data size: {X_combined.shape[0]} points")
    print(f"  Output range: [{y_combined.min():.6f}, {y_combined.max():.6f}]  mean={y_combined.mean():.6f}")

    # =========================
    # Linear Regression guide
    # =========================
    model = LinearRegression()
    try:
        model.fit(X_combined, y_combined)
    except Exception as e:
        # Jika gagal (mis. singular), tambahkan sedikit noise pada X agar invertible
        X_perturbed = X_combined + 1e-9 * np.random.randn(*X_combined.shape)
        model.fit(X_perturbed, y_combined)

    # Kandidat acak di [0,1]^dim, pilih prediksi tertinggi (eksploitasi ringan)
    candidates = np.random.uniform(0, 1, size=(n_candidates, dim))
    preds = model.predict(candidates)
    best_idx = int(np.argmax(preds))
    best_query = np.clip(candidates[best_idx], 0.0, 1.0)

    query_str = "-".join([f"{x:.6f}" for x in best_query])
    print(f"  Best predicted output: {preds[best_idx]:.6f}")
    print(f"  Query to submit     : {query_str}")

    queries_out.append(f"Function {i}: {query_str}")

# =========================
# SAVE ALL QUERIES TO FILE
# =========================
with open("week2_queries.txt", "w") as f:
    f.write("\n".join(queries_out))

print("\nAll queries saved to week2_queries.txt")


ValueError: Expect 8 inputs and 8 outputs, got 8 inputs and 16 outputs.

In [12]:
with open("week1/outputs.txt", "r") as f:
    text = f.read()

print("Isi outputs.txt:")
print(text)
print("\nSemua angka yang terdeteksi:")
import re
nums = re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", text)
print(nums)
print(f"\nJumlah angka terdeteksi: {len(nums)}")

Isi outputs.txt:
[np.float64(3.1124304418868536e-71), np.float64(0.3427580116095451), np.float64(-0.07865180298183852), np.float64(-4.448926246215535), np.float64(149.78214292831024), np.float64(-1.08635496159916), np.float64(0.01367815347703966), np.float64(9.5846559148596)]

Semua angka yang terdeteksi:
['64', '3.1124304418868536e-71', '64', '0.3427580116095451', '64', '-0.07865180298183852', '64', '-4.448926246215535', '64', '149.78214292831024', '64', '-1.08635496159916', '64', '0.01367815347703966', '64', '9.5846559148596']

Jumlah angka terdeteksi: 16


In [14]:
import numpy as np
import os
import re
import ast
from sklearn.linear_model import LinearRegression

# =========================
# CONFIGURATION
# =========================
base_path = "data/"             # ubah sesuai lokasi folder function_1 ... function_8
inputs_txt = "week1/inputs.txt"       # file dari email minggu lalu
outputs_txt = "week1/outputs.txt"     # file dari email minggu lalu
n_candidates = 1000             # jumlah titik acak untuk eksploitasi
save_queries = True             # simpan hasil ke week2_queries.txt

# =========================
# LOAD HELPERS
# =========================
def load_inputs(path):
    """Baca inputs.txt yang berisi array([...])"""
    with open(path, "r") as f:
        text = f.read().strip()

    # Hilangkan "array(" dan ")" agar jadi list of lists biasa
    text_clean = re.sub(r'array\(', '', text)
    text_clean = text_clean.replace(')', '')

    # Evaluasi menjadi list Python
    data = ast.literal_eval(text_clean)
    vectors = [np.array(x, dtype=float) for x in data]

    if len(vectors) != 8:
        print(f"Warning: Expected 8 inputs, found {len(vectors)}")
    return vectors

def load_outputs(path):
    """Baca outputs.txt yang berisi np.float64(...)"""
    with open(path, "r") as f:
        text = f.read().strip()

    # Bersihkan np.float64(...) agar jadi list angka biasa
    text_clean = text.replace("np.float64(", "").replace(")", "")

    # Evaluasi ke list Python
    data = ast.literal_eval(text_clean)
    outputs = np.array(data, dtype=float)

    if len(outputs) != 8:
        print(f"Warning: Expected 8 outputs, found {len(outputs)}")
    return outputs

# =========================
# LOAD WEEK-1 DATA FROM EMAIL
# =========================
new_inputs = load_inputs(inputs_txt)
new_outputs = load_outputs(outputs_txt)

print(f"✅ Loaded {len(new_inputs)} inputs and {len(new_outputs)} outputs from email")

# =========================
# PROCESS EACH FUNCTION
# =========================
queries_out = []

for i in range(1, 9):
    folder = os.path.join(base_path, f"function_{i}")
    input_file = os.path.join(folder, "initial_inputs.npy")
    output_file = os.path.join(folder, "initial_outputs.npy")

    # Load data awal
    X_init = np.load(input_file)
    y_init = np.load(output_file)

    # Gabungkan dengan hasil minggu lalu
    X_combined = np.vstack([X_init, new_inputs[i - 1].reshape(1, -1)])
    y_combined = np.append(y_init, new_outputs[i - 1])

    # Simpan versi updated
    np.save(os.path.join(folder, "updated_inputs.npy"), X_combined)
    np.save(os.path.join(folder, "updated_outputs.npy"), y_combined)

    dim = X_combined.shape[1]
    print(f"\n=== Function {i} ({dim}D) ===")
    print(f"  Data size: {X_combined.shape[0]} points")
    print(f"  Output range: [{y_combined.min():.6f}, {y_combined.max():.6f}]  mean={y_combined.mean():.6f}")

    # =========================
    # TRAIN LINEAR REGRESSION MODEL
    # =========================
    model = LinearRegression()
    try:
        model.fit(X_combined, y_combined)
    except Exception as e:
        # Jika error (misal singular matrix), tambahkan noise kecil
        X_perturbed = X_combined + 1e-9 * np.random.randn(*X_combined.shape)
        model.fit(X_perturbed, y_combined)

    # =========================
    # GENERATE CANDIDATE POINTS
    # =========================
    candidates = np.random.uniform(0, 1, size=(n_candidates, dim))
    preds = model.predict(candidates)

    best_idx = int(np.argmax(preds))
    best_query = np.clip(candidates[best_idx], 0.0, 1.0)

    query_str = "-".join([f"{x:.6f}" for x in best_query])
    print(f"  Best predicted output: {preds[best_idx]:.6f}")
    print(f"  Query to submit     : {query_str}")

    queries_out.append(f"Function {i}: {query_str}")

# =========================
# SAVE ALL QUERIES
# =========================
if save_queries:
    with open("week2_queries.txt", "w") as f:
        f.write("\n".join(queries_out))
    print("\n💾 All queries saved to week2_queries.txt")

print("\n✅ DONE — Copy each 'Query to submit' into the capstone portal.")

✅ Loaded 8 inputs and 8 outputs from email

=== Function 1 (2D) ===
  Data size: 11 points
  Output range: [-0.003606, 0.000000]  mean=-0.000328
  Best predicted output: 0.000201
  Query to submit     : 0.006394-0.019251

=== Function 2 (2D) ===
  Data size: 11 points
  Output range: [-0.065624, 0.611205]  mean=0.240863
  Best predicted output: 0.617061
  Query to submit     : 0.998753-0.057963

=== Function 3 (3D) ===
  Data size: 16 points
  Output range: [-0.398926, -0.034835]  mean=-0.105385
  Best predicted output: -0.022266
  Query to submit     : 0.975213-0.547548-0.017331

=== Function 4 (4D) ===
  Data size: 31 points
  Output range: [-32.625660, -4.025542]  mean=-16.826018
  Best predicted output: -4.773792
  Query to submit     : 0.073257-0.021068-0.573624-0.025056

=== Function 5 (4D) ===
  Data size: 21 points
  Output range: [0.112940, 1088.859618]  mean=151.200936
  Best predicted output: 748.658914
  Query to submit     : 0.039326-0.875323-0.917494-0.936770

=== Functio